In [2]:
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes
!pip install pandas matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 7.4 MB/s eta 0:00:00


In [2]:
import os
path_to_dataset = ""
#Per comodità, per capire se stiamo eseguendo su Google Colab o su Kaggle
if os.path.exists('/content/'):
        path_to_dataset = "/content/train_v2_drcat_02.csv"

    # Verifica se è presente la cartella "input" caratteristica di Kaggle
if os.path.exists('/kaggle/input/'):
        path_to_dataset = "/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv"


In [3]:
import pandas as pd
df = pd.read_csv(path_to_dataset)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44868 entries, 0 to 44867
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   text           44868 non-null  object
 1   label          44868 non-null  int64 
 2   prompt_name    44868 non-null  object
 3   source         44868 non-null  object
 4   RDizzl3_seven  44868 non-null  bool  
dtypes: bool(1), int64(1), object(3)
memory usage: 1.4+ MB


### Creazione di train e test

In [4]:
df = df.drop('source',axis = 1)
df = df.drop('RDizzl3_seven',axis = 1)
data = df
data = data.drop('label',axis = 1)
data = data.drop('prompt_name',axis = 1)

In [5]:
data

,text
0,Phones\n\nModern humans today are always on th...
1,This essay will explain if drivers should or s...
2,Driving while the use of cellular devices\n\nT...
3,Phones & Driving\n\nDrivers should not be able...
4,Cell Phone Operation While Driving\n\nThe abil...
...,...
44863,"Dear Senator,\n\nI am writing to you today to ..."
44864,"Dear Senator,\n\nI am writing to you today to ..."
44865,"Dear Senator,\n\nI am writing to you today to ..."
44866,"Dear Senator,\n\nI am writing to you today to ..."


#### Data Cleaning

Togliamo le stop-words.


In [6]:
import tensorflow as tf

#import nltk
#from nltk.corpus import stopwords
#NLTK
#nltk.download('stopwords')
#stop = set(stopwords.words('english'))

#def remove_stop_words_nltk(text):
#    words = text.split()
#    filtered_words = [word for word in words if word.lower() not in stop]
#    return ' '.join(filtered_words)
#Alternativa con SpaCy
import spacy
#loading the english language small model of spacy
en = spacy.load('en_core_web_sm')
sw_spacy = en.Defaults.stop_words
print(sw_spacy)

def remove_stop_words_spacy(text):
    words = [word for word in text.split() if word.lower() not in sw_spacy]
    new_text = " ".join(words)
    return new_text

#Alternativa con Gensim
#import gensim
#from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS
#print(STOPWORDS)

#def remove_stop_words_gensim(text):
#  new_text = remove_stopwords(text)
#  return new_text

{'anything', 'thereby', 'under', 'another', 'hundred', 'myself', 'has', 'twelve', 'still', 'therefore', 'became', '’m', "'d", 'otherwise', 'therein', 'yours', 'empty', 'whereby', 'out', 'hers', 'every', 'again', 'fifteen', '‘d', 'many', 'twenty', 'done', 'whereafter', 'latterly', 'back', 'six', 'if', 'n’t', 'whether', 'does', 'what', 'seem', 'whose', 'from', 'they', 'ourselves', 'show', 'each', 'such', 'and', 'down', '‘re', 'eight', 'doing', 'who', 'n‘t', 'moreover', 'somewhere', 'various', 'anyone', 'someone', 'how', 'regarding', 'after', 'rather', 'whole', 'several', 'none', 'do', 'will', 'else', 'really', 'which', 'cannot', 'namely', 'would', 'around', 'was', 'him', 'whence', 'all', 'everywhere', 'together', "'re", 'forty', 'thence', 'were', 'further', 'other', 'quite', 'becoming', 'keep', 'too', 'alone', 'side', 'below', 'had', 'us', 'no', 'me', 'anyhow', 'sometimes', 'something', 'until', 'always', 'been', 'whereas', 'i', 'above', 'on', 'off', 'through', 'toward', 'give', 'anyway'

In [7]:
for index,row in data.iterrows():
  #print(f"{index} {row}")
  new_string = remove_stop_words_spacy(row['text'])
  data.at[index,'text'] = new_string


data.head(1)

,text
0,Phones Modern humans today phone. phone 5 hour...


Togliamo la punteggiatura e i caratteri speciali

In [8]:
import string

def remove_punctuation_with_string(text):
    new_text = ""
    for word in text:
      for i in range(len(word)):
        if word[i] not in string.punctuation:
          new_text+=word[i]
    return new_text

In [9]:
for index,row in data.iterrows():
  #print(f"{index} {row}")
  new_string = remove_punctuation_with_string(row['text'])
  data.at[index,'text'] = new_string


data.head(1)

,text
0,Phones Modern humans today phone phone 5 hours...


#### Some tries

In [ ]:
lunghezza_massima = max(len(sequenza.split()) for sequenza in data['text'])

print("Lunghezza massima delle sequenze:", lunghezza_massima)

In [ ]:
import math
# Calcola la lunghezza media delle sequenze nel dataset
lunghezza_medio = sum(len(sequenza.split()) for sequenza in data['text']) / len(data['text'])
lunghezza_medio = math.ceil(lunghezza_medio)
print("Lunghezza media delle sequenze:", lunghezza_medio)

In [ ]:
import numpy as np
# Calcola la lunghezza delle sequenze
lunghezze_sequenze = [len(sequenza.split()) for sequenza in data['text']]

# Scegli il percentile desiderato (ad esempio, il 95° percentile)
percentile_desiderato = 95

# Calcola il valore del percentile
lunghezza_percentile = np.percentile(lunghezze_sequenze, percentile_desiderato)

lunghezza_percentile =  math.ceil(lunghezza_percentile)

print(f"{percentile_desiderato}° percentile della lunghezza delle sequenze:", lunghezza_percentile)

#### Train_test_split

In [10]:
from sklearn.model_selection import train_test_split

# Supponiamo che il tuo dataset sia rappresentato da X (feature) e y (etichette)
#X_train, X_temp, y_train, y_temp = train_test_split(data['text'], df['label'], test_size=0.2, random_state=42)
#X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [11]:
train_input,val_input,train_label,val_label = train_test_split(data['text'],df['label'])


In [ ]:
print(train_input.shape,train_label.shape)

#### Bert Tokenizer

In [12]:
from transformers import BertTokenizer,TFBertModel,BertConfig,TFBertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [13]:
to_print = data['text'][0]
print(to_print)

Phones Modern humans today phone phone 5 hours day stop All text forward group Chats social media driving bad consequences stuff happens comes phone certain areas United States ban phones class rooms it people phones know certain apps Apps like Facebook Twitter Instagram Snapchat like friend moves away want contact contact posting videos text messages People different ways communicate phone Phones changed generation Driving way around People phones it cause Problems Thats theres thing thats called texting driving Thats important thing remember people think Its stupid matter obey thats way save news accident suicide involve looking theyre going tweet sent injury death mysterious number says Im going kill know live dont know persons contact It makes puzzled start freak out end badly Phones fine use its best way come help problem cant find help always phone you phones day long youre safe come use trouble sure like phone youre middle driving news updated people stupid involves phones safes

In [14]:
to_print_tokenized = tokenizer.tokenize(to_print)
print(to_print_tokenized)

['phones', 'modern', 'humans', 'today', 'phone', 'phone', '5', 'hours', 'day', 'stop', 'all', 'text', 'forward', 'group', 'chat', '##s', 'social', 'media', 'driving', 'bad', 'consequences', 'stuff', 'happens', 'comes', 'phone', 'certain', 'areas', 'united', 'states', 'ban', 'phones', 'class', 'rooms', 'it', 'people', 'phones', 'know', 'certain', 'apps', 'apps', 'like', 'facebook', 'twitter', 'ins', '##tagram', 'snap', '##cha', '##t', 'like', 'friend', 'moves', 'away', 'want', 'contact', 'contact', 'posting', 'videos', 'text', 'messages', 'people', 'different', 'ways', 'communicate', 'phone', 'phones', 'changed', 'generation', 'driving', 'way', 'around', 'people', 'phones', 'it', 'cause', 'problems', 'that', '##s', 'there', '##s', 'thing', 'that', '##s', 'called', 'text', '##ing', 'driving', 'that', '##s', 'important', 'thing', 'remember', 'people', 'think', 'its', 'stupid', 'matter', 'obey', 'that', '##s', 'way', 'save', 'news', 'accident', 'suicide', 'involve', 'looking', 'they', '##r

In [15]:
print('Token to ids',tokenizer.convert_tokens_to_ids(tokenizer.tokenize(to_print)))

Token to ids [11640, 2715, 4286, 2651, 3042, 3042, 1019, 2847, 2154, 2644, 2035, 3793, 2830, 2177, 11834, 2015, 2591, 2865, 4439, 2919, 8465, 4933, 6433, 3310, 3042, 3056, 2752, 2142, 2163, 7221, 11640, 2465, 4734, 2009, 2111, 11640, 2113, 3056, 18726, 18726, 2066, 9130, 10474, 16021, 23091, 10245, 7507, 2102, 2066, 2767, 5829, 2185, 2215, 3967, 3967, 14739, 6876, 3793, 7696, 2111, 2367, 3971, 10639, 3042, 11640, 2904, 4245, 4439, 2126, 2105, 2111, 11640, 2009, 3426, 3471, 2008, 2015, 2045, 2015, 2518, 2008, 2015, 2170, 3793, 2075, 4439, 2008, 2015, 2590, 2518, 3342, 2111, 2228, 2049, 5236, 3043, 15470, 2008, 2015, 2126, 3828, 2739, 4926, 5920, 9125, 2559, 2027, 2890, 2183, 1056, 28394, 2102, 2741, 4544, 2331, 8075, 2193, 2758, 10047, 2183, 3102, 2113, 2444, 2123, 2102, 2113, 5381, 3967, 2009, 3084, 14909, 2707, 11576, 2041, 2203, 6649, 11640, 2986, 2224, 2049, 2190, 2126, 2272, 2393, 3291, 2064, 2102, 2424, 2393, 2467, 3042, 2017, 11640, 2154, 2146, 2115, 2063, 3647, 2272, 2224, 4390,

In [43]:
max_len = 0
for d in data['text']:
  input_ids = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(d))
  max_len = max(max_len,len(input_ids))
print(max_len)
max_len=512

KeyboardInterrupt: ignored

In [44]:
max_len=512

In [45]:
def build_inputs_for_bert(data,max_len):
  input_ids = []
  attention_masks = []
  i = 0
  for d in data:
    if(i<3):
      print("Data",d)
      print()
    encoded_dict = tokenizer.encode_plus(d,add_special_tokens=True,max_length=max_len,truncation=True,return_attention_mask = True)
    if(i<3):
      print("dict",encoded_dict['input_ids'])
      print()
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])
    i=i+1
  input_ids = tf.convert_to_tensor(input_ids)
  attention_masks = tf.convert_to_tensor(attention_masks)
  return input_ids,attention_masks

(1) Tokenize the sentence.
(2) Prepend the ‘[CLS]’ token to the start.
(3) Append the ‘[SEP]’ token to the end.
(4) Map tokens to their IDs.
(5) Pad or truncate the sentence to ‘max_length’
(6) Create attention masks for ‘[PAD]’ tokens.

In [ ]:
input_ids2 = []
attention_masks2 = []
for i in range(len(val_input)):
  print(f'{val_input[0]}')
  encoded_dict = tokenizer.encode_plus(val_input[i],add_special_tokens=True,padding='max_length',return_attention_mask = True)
  input_ids2.append(encoded_dict['input_ids'])
  attention_masks2.append(encoded_dict['attention_mask'])
  if i==1:
    break
input_ids2 = tf.convert_to_tensor(input_ids2)
attention_masks2 = tf.convert_to_tensor(attention_masks2)

In [ ]:
train_inp,train_mask = build_inputs_for_bert(train_input,max_len)

Data The Power Imagination Imagination powerful tool ability shape world ways thought possible driving force innovation progress it technological advancements enjoy today compelling arguments importance imagination fact led greatest inventions human history Wright Brothers invention airplane discovery penicillin power imagination allowed overcome seemingly insurmountable challenges push boundaries possible Imagination plays crucial role personal lives allows envision better future loved ones work achieving goals use imagination visualize success likely action reality Furthermore imagination power heal bring joy lives feeling stressed use imagination transport happier place its tropical beach peaceful forest help feel relaxed centered provide temporary escape problems conclusion imagination powerful tool ability shape world countless ways driving force innovation progress personal growth essential human experience So lets embrace imagination use create brighter beautiful future generati

In [21]:
val_inp,val_mask = build_inputs_for_bert(val_input,max_len)

Data Hey there So know meet new know youre gonna BFFs not Yeah impressions real thing man totally break relationship Like think it meet someone youre like judging based look talk say dont like see well its kinda hard forget it Its like brain like Hmm person kinda weird dont wanna hang them dont thing falls apart But hand good impression its like rainbows unicorns Youre like Oh gosh person cool wanna hang time do its like best thing ever mean think it met like super nice friendly found actually kinda mean something Its like whoa impressions important man tell lot person its people either Like think saw classroom like it think was like super lame mean personally think its pretty cool Im sure people agree Anyway point is impressions big deal break relationships affect feel place thing So time meet new walk new space try aware impression know change everything Oh thing

dict [101, 4931, 2045, 2061, 2113, 3113, 2047, 2113, 2115, 2063, 6069, 28939, 10343, 2025, 3398, 19221, 2613, 2518, 2158,

In [22]:
train_label = tf.convert_to_tensor(train_label)

In [23]:
val_label = tf.convert_to_tensor(val_label)

In [24]:
print("train_input_shape",train_inp.shape)
print("train_mask_shape",train_mask.shape)
print("val_input_shape",val_inp.shape)
print("val_mask_shape",val_mask.shape)
print("train_label_shape",train_label.shape)
print("val_label_shape",val_label.shape)


train_input_shape (33651, 1733)
train_mask_shape (33651, 1733)
val_input_shape (11217, 1733)
val_mask_shape (11217, 1733)
train_label_shape (33651,)
val_label_shape (11217,)


In [38]:
!pip install --upgrade transformers tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 636.7 kB/s eta 0:00:00
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.0
    Uninstalling tensorflow-2.15.0:
      Successfully uninstalled tensorflow-2.15.0


In [40]:
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2,ignore_mismatched_sizes=True)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
import tensorflow as tf
import keras
log_dir = "/content/log"
model_save_path="/content/log/bert_model.h5"

callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
print("Summary",bert_model.summary())

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) #BinaryCrossEntropy
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)
bert_model.compile(loss=loss,optimizer=optimizer,metrics=[metric])

#bert_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Model: "tf_bert_for_sequence_classification_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_191 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Summary None


In [42]:
history = bert_model.fit([train_inp,train_mask],train_label,batch_size=32,epochs=4,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)

Epoch 1/4


InvalidArgumentError: ignored

### Create text encoder


In [ ]:
#VOCAB_SIZE = 1000 #Parametro da sistemare

#encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE) #output_sequence_length=50 in alternativa ad input_length
#encoder.adapt(X_train.values.flatten())

##### Controlliamo se l'encoder funziona

In [ ]:
#import numpy as np
#vocab = np.array(encoder.get_vocabulary())
#vocab[:20]

### Applichiamo il Padding (VEDERE MEGLIO COME FARE QUESTA COSA)


In [ ]:
#from tensorflow.keras.preprocessing.sequence import pad_sequences
#X_train = pad_sequences(X_train, padding='post', maxlen=100)

In [ ]:
#X_train

### Creiamo un embedding

In [ ]:
embedding_layer = tf.keras.layers.Embedding(1000, 5) #SERVE SOLO DA ESEMPIO IL VERO PADDING è NELLA RETE

In [ ]:
#TEST PER VEDERE SE FUNZIONA IL LAYER EMBEDDING #SERVE SOLO DA ESEMPIO IL VERO PADDING è NELLA RETE
result = embedding_layer(tf.constant([1, 2, 3]))
result.numpy()

## Creazione modello

In [ ]:
#model = tf.keras.Sequential([
#    encoder, #TextVectorization
#    tf.keras.layers.Embedding( #Embedding
#        #gestire anche dimensione input e output
#        input_dim=len(encoder.get_vocabulary()),
#        output_dim=3,
#        # aggiungere input_length  =  lunghezza massima o come media o come percentile 95%
#        input_length = lunghezza_massima,
#        mask_zero=True),
 #   tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),#
#    tf.keras.layers.Dense(64, activation='relu'),
#    tf.keras.layers.Dense(1, activation='sigmoid')
#])

In [ ]:
#model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val))

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score



y_pred = model.predict(X_test)
y_pred = y_pred.astype(int)
y_test = y_test.astype(int)

# Stampa il classification report e l'accuracy score
report = classification_report(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(report)
print("****")
print(accuracy)




In [ ]:
print(report)
print("****")
print(accuracy)

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)

In [ ]:
for key in X_test :
    predictions = model.predict(np.array([key]))
    print(predictions)
    break

In [ ]:
for key in y_test :
    print(key)
    break